In [1]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 5.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset
import sacrebleu

In [3]:
df = pd.read_csv("/kaggle/input/mansi-russian-parralel-corpus/overall_80K.csv")

In [4]:
df

,Unnamed: 0,target,source
0,0,Та пыгрисит маим вармаль э̄рнэ поратэт ат верм...,Те мальчики не выполнят задание в назначенный ...
1,1,"Ха̄йтыматэ тӯр ва̄тан ёхтыс, вит ва̄тан ха̄йтыс.","Бегая к берегу озера пришла, к воде подбежала."
2,2,Вит са̄мыл сунсым о̄нтыс,Вода прибывала на глазах
3,3,"Атаявев, акваг лылынг тагл ворн та тотавев.","Обнюхивает нас, живыми на кладбище уносит."
4,4,"Ман ты пӣлтал, веськат хумиюв нэтхуньт ат ёр...",Мы никогда не забудем этого честного человека.
...,...,...,...
81141,2345,А̄нумн ка̄салахты аквтуп тамле о̄лнэ накыт ма̄...,"Мне кажется, что подобные случаи могут вызыват..."
81142,2346,А̄танэ нё̄тнэ̄г юил акван-атманэ.,Волосы аккуратно собраны сзади.
81143,2347,"Тох тай, культура сака тэ̄пгалан мед а̄тим.","В общем, культуры интенсивного потребления мед..."
81144,2348,"Тувыл Уэйтс ты музыкантыг ёт, Чарли Рич ос Фрэ...",Затем Уэйтс отправился на гастроли с такими му...


In [5]:
# чистим дф от мусора
df = df.drop(columns=[df.columns[0]])

In [6]:
df

,target,source
0,Та пыгрисит маим вармаль э̄рнэ поратэт ат верм...,Те мальчики не выполнят задание в назначенный ...
1,"Ха̄йтыматэ тӯр ва̄тан ёхтыс, вит ва̄тан ха̄йтыс.","Бегая к берегу озера пришла, к воде подбежала."
2,Вит са̄мыл сунсым о̄нтыс,Вода прибывала на глазах
3,"Атаявев, акваг лылынг тагл ворн та тотавев.","Обнюхивает нас, живыми на кладбище уносит."
4,"Ман ты пӣлтал, веськат хумиюв нэтхуньт ат ёр...",Мы никогда не забудем этого честного человека.
...,...,...
81141,А̄нумн ка̄салахты аквтуп тамле о̄лнэ накыт ма̄...,"Мне кажется, что подобные случаи могут вызыват..."
81142,А̄танэ нё̄тнэ̄г юил акван-атманэ.,Волосы аккуратно собраны сзади.
81143,"Тох тай, культура сака тэ̄пгалан мед а̄тим.","В общем, культуры интенсивного потребления мед..."
81144,"Тувыл Уэйтс ты музыкантыг ёт, Чарли Рич ос Фрэ...",Затем Уэйтс отправился на гастроли с такими му...


In [7]:
train_df, test_val_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(test_val_df, test_size=0.5, random_state=42)

In [8]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [9]:
tokenizer = AutoTokenizer.from_pretrained("tartuNLP/m2m100_418M_smugri")
model = AutoModelForSeq2SeqLM.from_pretrained("tartuNLP/m2m100_418M_smugri")

tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/931 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

In [10]:
tokenizer.add_special_tokens({'additional_special_tokens': ['<mns_MNS>']})
model.resize_token_embeddings(len(tokenizer))

M2M100ScaledWordEmbedding(128109, 1024, padding_idx=1)

In [11]:
def preprocess_function(examples):
    inputs = tokenizer(examples['source'], truncation=True, padding='max_length', max_length=128)
    targets = tokenizer(examples['target'], truncation=True, padding='max_length', max_length=128)
    inputs['labels'] = targets['input_ids']
    return inputs

In [12]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/64916 [00:00<?, ? examples/s]

Map:   0%|          | 0/8115 [00:00<?, ? examples/s]

Map:   0%|          | 0/8115 [00:00<?, ? examples/s]

In [13]:
train_dataset = train_dataset.remove_columns(['source', 'target', '__index_level_0__'])
val_dataset = val_dataset.remove_columns(['source', 'target', '__index_level_0__'])
test_dataset = test_dataset.remove_columns(['source', 'target', '__index_level_0__'])

In [14]:
import torch
torch.cuda.empty_cache()

In [16]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./m2m_smugri_finetuned",
    evaluation_strategy="no", 
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4, 
    weight_decay=0.01,
    save_total_limit=5, 
    save_strategy="no", 
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir="./logs",
    report_to="none", 
    fp16=True, 
    load_best_model_at_end=False,  
)

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    bleu = sacrebleu.corpus_bleu(decoded_preds, [decoded_labels])
    chrf = sacrebleu.corpus_chrf(decoded_preds, [decoded_labels])

    return {"bleu": bleu.score, "chrf": chrf.score}

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

Step,Training Loss
500,0.815200
1000,0.605300
1500,0.520200
2000,0.483900
2500,0.439800
3000,0.421800
3500,0.409100
4000,0.400500
4500,0.365800
5000,0.359900


TrainOutput(global_step=12171, training_loss=0.3826604242460385, metrics={'train_runtime': 15632.2783, 'train_samples_per_second': 12.458, 'train_steps_per_second': 0.779, 'total_flos': 5.27516602246103e+16, 'train_loss': 0.3826604242460385, 'epoch': 2.999815145726785})

In [20]:
model.save_pretrained("./m2m_smugri_finetuned_checkpoint")
tokenizer.save_pretrained("./m2m_smugri_finetuned_checkpoint")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}


In [29]:
model.eval()

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): M2M100ScaledWordEmbedding(128109, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): M2M100ScaledWordEmbedding(128109, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-11): 12 x M2M100EncoderLayer(
          (self_attn): M2M100Attention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
       

In [30]:
import random 
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

random.seed(42) 
random_indices = random.sample(range(len(test_dataset)), 1000)
random_test_samples = [test_dataset[i] for i in random_indices]

def generate_translation(sample):
    input_ids = torch.tensor(sample['input_ids']).unsqueeze(0).to(device)  # Конвертируем список в тензор
    with torch.no_grad():
        generated_ids = model.generate(input_ids, max_length=200, num_beams=5, early_stopping=True)
    return tokenizer.decode(generated_ids[0], skip_special_tokens=True)

original_texts = []
correct_translations = []
model_translations = []

for sample in tqdm(random_test_samples):
    input_text = tokenizer.decode(sample['input_ids'], skip_special_tokens=True)
    correct_translation = tokenizer.decode(sample['labels'], skip_special_tokens=True)
    model_translation = generate_translation(sample)

    original_texts.append(input_text)
    correct_translations.append(correct_translation)
    model_translations.append(model_translation)
    
df_results = pd.DataFrame({
    "Original Text": original_texts,
    "Correct Translation": correct_translations,
    "Model Translation": model_translations
})

100%|██████████| 1000/1000 [13:12<00:00,  1.26it/s]


In [31]:
df_results.head()

,Original Text,Correct Translation,Model Translation
0,__en__ Отсюда подальше уходи.,__en__ Тыгыл э̄ла минэн.,__en__ Тыгыл э̄лаль минэн.
1,"__en__ Этот дом с осени работает, и дети сейча...","__en__ Ты кол таквсы пораныл рпиты, ос ня̄врам...","__en__ Ты кол таквсы па̄сныл рпиты, ос ня̄врам..."
2,__en__ Его во время войны снайпером звали. И с...,__en__ Хо̄нт порат тав снайперыг ла̄ввес... Нё...,__en__ Тав хо̄нтлын порат снайперыг ла̄ввес. Н...
3,__en__ Такое собрание в нашем округе впервые п...,__en__ Тамле сапра̄ни ты та̄л ма̄н округувт о̄...,__en__ Тамле сапра̄ни ма̄н округувт о̄выл щёс ...
4,__en__ И отправился мальчик с ними в дорогу.,__en__ Пыгрись ос та̄н ётаныл минмыгтас.,__en__ Пыгрись та̄н ётаныл лё̄хын минас.


In [32]:
df_results.loc[:, 'Original Text'] = df_results['Original Text'].str.replace('__en__', '')
df_results.loc[:, 'Correct Translation'] = df_results['Correct Translation'].str.replace('__en__', '')
df_results.loc[:, 'Model Translation'] = df_results['Model Translation'].str.replace('__en__', '')
    
bleu_score = sacrebleu.corpus_bleu(df_results['Model Translation'].tolist(), 
                                   [df_results['Correct Translation'].tolist()]).score
chrf_score = sacrebleu.corpus_chrf(df_results['Model Translation'].tolist(), 
                                   [df_results['Correct Translation'].tolist()]).score

print(f"BLEU Score: {bleu_score}")
print(f"ChrF Score: {chrf_score}")

df_results.head(50)

BLEU Score: 19.11751671599327
ChrF Score: 50.052403383616465


,Original Text,Correct Translation,Model Translation
0,Отсюда подальше уходи.,Тыгыл э̄ла минэн.,Тыгыл э̄лаль минэн.
1,"Этот дом с осени работает, и дети сейчас так ...","Ты кол таквсы пораныл рпиты, ос ня̄врамыт ань...","Ты кол таквсы па̄сныл рпиты, ос ня̄врамыт ань..."
2,Его во время войны снайпером звали. И соболь ...,Хо̄нт порат тав снайперыг ла̄ввес... Нёхыс ат...,Тав хо̄нтлын порат снайперыг ла̄ввес. Нёхс ат...
3,Такое собрание в нашем округе впервые подгото...,Тамле сапра̄ни ты та̄л ма̄н округувт о̄выл щё...,Тамле сапра̄ни ма̄н округувт о̄выл щёс щё̄пит...
4,И отправился мальчик с ними в дорогу.,Пыгрись ос та̄н ётаныл минмыгтас.,Пыгрись та̄н ётаныл лё̄хын минас.
5,Первую неделю дети учились готовить националь...,О̄выл са̄тыт а̄гирищит-пыгрищит тэ̄нут ва̄рук...,О̄выл са̄т ня̄врамыт лима̄ мир тэ̄нут па̄йтук...
6,"Два моих сына оба в армии служили, в городе М...","Пыгагум сас армиян ялсы̄г, Муравленко ст хани...","Кит пыгагум сас армият лусытасыт, Муравленко ..."
7,"Я девушка манси, меня зовут Полина Албина.","Ам ма̄ньщи а̄ги, намум Полина Албина.","Ам ма̄ньщи а̄ги, ам намум Полина Албина."
8,К дому так подошли.,Колн ты ёхтысыг.,Колн тох ёхтысы̄г.
9,"Они говорили, что готовы умереть.","Ла̄всасыт, эрттум холукв ка̄саще̄гыт.","Та̄н ла̄всыт, та̄н а̄тимыг ēмтсыт."


In [23]:
import shutil
shutil.make_archive("m2m_smugric_finetuned", 'zip', "/kaggle/working/m2m_smugri_finetuned_checkpoint")

'/kaggle/working/m2m_smugric_finetuned.zip'

In [24]:
from IPython.display import FileLink

FileLink('m2m_smugric_finetuned.zip')

/kaggle/working/m2m_smugric_finetuned.zip

1